In [16]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [2]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [3]:
df_store.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


In [22]:
df1 = pd.merge(df_receipt,df_store,how='inner')
df1.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
1,20181116,1258329600,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
2,20170118,1200614400,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
3,20190524,1274659200,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0
4,20190419,1271635200,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店,14,神奈川県,神奈川県横浜市都筑区葛が谷,カナガワケンヨコハマシツヅキククズガヤ,045-123-4031,139.5633,35.53573,1886.0


In [23]:
label = df_receipt.columns.values
label = np.append(label,'store_name')
label

array(['sales_ymd', 'sales_epoch', 'store_cd', 'receipt_no',
       'receipt_sub_no', 'customer_id', 'product_cd', 'quantity',
       'amount', 'store_name'], dtype=object)

In [24]:
df1[label].head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1258329600,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1200614400,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1274659200,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1271635200,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店
5,20181119,1258588800,S14006,1152,2,ZZ000000000000,P050701001,1,88,葛が谷店
6,20171211,1228953600,S14006,1132,2,CS006515000175,P090903001,1,80,葛が谷店
7,20191021,1287619200,S14006,1112,2,CS006415000221,P040602001,1,405,葛が谷店
8,20170710,1215648000,S14006,1132,2,CS006411000036,P090301051,1,330,葛が谷店
9,20190805,1280966400,S14006,112,1,CS006211000012,P050104001,1,115,葛が谷店


P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [27]:
df_product.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0
3,P040101004,4,401,40101,248.0,186.0
4,P040101005,4,401,40101,268.0,201.0


In [28]:
df_category.head()

,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
0,4,惣菜,401,御飯類,40101,弁当類
1,4,惣菜,401,御飯類,40102,寿司類
2,4,惣菜,402,佃煮類,40201,魚介佃煮類
3,4,惣菜,402,佃煮類,40202,海草佃煮類
4,4,惣菜,402,佃煮類,40203,野菜佃煮類


In [30]:
df2 = pd.merge(df_product,df_category,how='inner')
df2.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類


In [31]:
label2 = df_product.columns.values
label2 = np.append(label2,'category_small_name')
label2

array(['product_cd', 'category_major_cd', 'category_medium_cd',
       'category_small_cd', 'unit_price', 'unit_cost',
       'category_small_name'], dtype=object)

In [32]:
df2[label2].head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,4,401,40101,198.0,149.0,弁当類
1,P040101002,4,401,40101,218.0,164.0,弁当類
2,P040101003,4,401,40101,230.0,173.0,弁当類
3,P040101004,4,401,40101,248.0,186.0,弁当類
4,P040101005,4,401,40101,268.0,201.0,弁当類
5,P040101006,4,401,40101,298.0,224.0,弁当類
6,P040101007,4,401,40101,338.0,254.0,弁当類
7,P040101008,4,401,40101,420.0,315.0,弁当類
8,P040101009,4,401,40101,498.0,374.0,弁当類
9,P040101010,4,401,40101,580.0,435.0,弁当類


P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [33]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [34]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [38]:
df3 = pd.merge(df_customer,df_receipt,on='customer_id',how='left')
df3.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,product_cd,quantity,amount
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,20170507.0,1.210118e+09,S13031,1102.0,1.0,P060103001,1.0,100.0
3,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,20171026.0,1.224979e+09,S13031,1182.0,1.0,P090203004,1.0,320.0
4,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,20190325.0,1.269475e+09,S13031,1192.0,1.0,P071401025,1.0,2400.0


In [51]:
df3 = df3.fillna(0)
df3 = df3.query('gender_cd == 1')
df3 = df3.query('not customer_id.str.startswith("Z")',engine='python')
df3.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,product_cd,quantity,amount
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,0.0,0.000000e+00,0,0.0,0.0,0,0.0,0.0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,20170507.0,1.210118e+09,S13031,1102.0,1.0,P060103001,1.0,100.0
3,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,20171026.0,1.224979e+09,S13031,1182.0,1.0,P090203004,1.0,320.0
4,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,20190325.0,1.269475e+09,S13031,1192.0,1.0,P071401025,1.0,2400.0
5,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,20170111.0,1.200010e+09,S13031,1132.0,2.0,P071203007,1.0,448.0


In [54]:
df3.groupby('customer_id').amount.sum().reset_index().head(10)

,customer_id,amount
0,CS001112000009,0.0
1,CS001112000019,0.0
2,CS001112000021,0.0
3,CS001112000023,0.0
4,CS001112000024,0.0
5,CS001112000029,0.0
6,CS001112000030,0.0
7,CS001113000004,1298.0
8,CS001113000010,0.0
9,CS001114000005,626.0


P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [64]:
df_receipt_Z = df_receipt.query('not customer_id.str.startswith("Z")',engine='python')
df_days = df_receipt_Z.groupby('customer_id').sales_ymd.count().reset_index().sort_values(by='sales_ymd',ascending=False).head(20)
df_days

,customer_id,sales_ymd
7986,CS040214000008,46
3488,CS015415000185,44
2429,CS010214000010,44
2428,CS010214000002,42
5908,CS028415000007,42
1951,CS007515000107,40
4693,CS021514000045,40
3682,CS016415000141,40
3899,CS017415000097,40
4883,CS022515000226,38


In [66]:
df_amount = df_receipt_Z.groupby('customer_id').amount.sum().reset_index().sort_values(by='amount',ascending=False).head(20)
df_amount

,customer_id,amount
3899,CS017415000097,23086
3488,CS015415000185,20153
6502,CS031414000051,19202
5908,CS028415000007,19127
332,CS001605000009,18925
2429,CS010214000010,18585
3682,CS016415000141,18372
1817,CS006515000023,18372
2681,CS011414000106,18338
7683,CS038415000104,17847


In [69]:
df4 = pd.merge(df_days,df_amount,how='outer')
df4

,customer_id,sales_ymd,amount
0,CS040214000008,46.0,NaN
1,CS015415000185,44.0,20153.0
2,CS010214000010,44.0,18585.0
3,CS010214000002,42.0,NaN
4,CS028415000007,42.0,19127.0
5,CS007515000107,40.0,NaN
6,CS021514000045,40.0,NaN
7,CS016415000141,40.0,18372.0
8,CS017415000097,40.0,23086.0
9,CS022515000226,38.0,NaN


P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [71]:
len(df_store) * len(df_product)

531590